In [245]:
%load_ext autoreload
%autoreload 2

import torch
import pandas as pd

from data_fetcher import PJMDataFetcher
from sliding_data_buffer import SlidingDataBuffer
from preprocess import clean_actual_data, clean_forecast_data, transform_data, inverse_transform_data
from forecast_data import create_my_records, agency_df_to_records, merge_records

from model import DA_RNN

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [246]:
lookback = 64
input_size = 3
hidden_size = 128
horizon = 24
num_layers = 3
lstm_dropout = 0.3
final_dropout = 0.1
lr = 1e-3
num_epochs = 15
interval_minutes = 5
device = 'cuda' if torch.cuda.is_available() else 'cpu'

model = DA_RNN(
    input_dim=input_size,
    hidden_dim=hidden_size,
    output_dim=1,
    horizon=horizon,
    lookback=lookback,
    encoder_layers=2,
    decoder_layers=2
)
model.load_state_dict(torch.load("artifacts/model.pt"))
model.to(device)
model.eval()

DA_RNN(
  (encoder_cells): ModuleList(
    (0): LSTMCell(3, 128)
    (1): LSTMCell(128, 128)
  )
  (input_attn): InputAttention(
    (W_e): Linear(in_features=257, out_features=64, bias=True)
    (v_e): Linear(in_features=64, out_features=1, bias=True)
  )
  (decoder_cells): ModuleList(
    (0): LSTMCell(1, 128)
    (1): LSTMCell(128, 128)
  )
  (temporal_attn): TemporalAttention(
    (W_d): Linear(in_features=384, out_features=64, bias=True)
    (v_d): Linear(in_features=64, out_features=1, bias=True)
  )
  (fc): Linear(in_features=320, out_features=1, bias=True)
)

In [247]:
def create_tensor(df):
    features = ['load', 'sin_time', 'cos_time']
    x_array = df[features].values
    x_tensor = torch.tensor(x_array, dtype=torch.float32).unsqueeze(0)
    y_hist = df["load"].values[-64:]
    y_hist_tensor = torch.tensor(y_hist, dtype=torch.float32).unsqueeze(0)
    return x_tensor, y_hist_tensor


In [248]:
fetcher = PJMDataFetcher(download_dir=r'D:\ML\EnergyPrediction\deploy\storage', headless=True)
historical_actual = fetcher.fetch_historical_actual()
forecast = fetcher.fetch_recent_forecast()

Filled form from 6/20/2025 02:05 to 6/20/2025 08:05


In [249]:
historical_actual_clean, latest_ts = clean_actual_data(historical_actual)
data = historical_actual_clean.tail(64)
working_buffer = SlidingDataBuffer(data)
data_for_model = transform_data(working_buffer.get_data(), r"artifacts/scalers.pkl")
if len(data_for_model) != 64:
    raise ValueError("Expected 64 rows for model input.")
x_tensor, y_hist_tensor = create_tensor(data_for_model)
x_tensor = x_tensor.to(device)
y_hist_tensor = y_hist_tensor.to(device)
with torch.no_grad():
    y_pred = model(x_tensor, y_hist_tensor)
y_pred = y_pred.squeeze().cpu().numpy()
y_pred_forecast = inverse_transform_data(y_pred, r"artifacts/scalers.pkl")




forecast_clean = clean_forecast_data(forecast, latest_ts=latest_ts)

In [257]:
mr = create_my_records(latest_ts, list(y_pred_forecast))
ar = agency_df_to_records(forecast_clean)
records = merge_records(mr, ar)
df_preds = pd.DataFrame([r.__dict__ for r in records])

In [258]:
df_preds

,issued_at,target_time,horizon,your_pred,agency_pred,actual
0,2025-06-20 12:05:00,2025-06-20 12:05:00,0,89674.483240,89573.0,None
1,2025-06-20 12:05:00,2025-06-20 12:10:00,1,89681.819675,89919.0,None
2,2025-06-20 12:05:00,2025-06-20 12:15:00,2,89733.865497,90263.0,None
3,2025-06-20 12:05:00,2025-06-20 12:20:00,3,89793.668643,90604.0,None
4,2025-06-20 12:05:00,2025-06-20 12:25:00,4,89863.459007,90947.0,None
5,2025-06-20 12:05:00,2025-06-20 12:30:00,5,89932.327840,91286.0,None
6,2025-06-20 12:05:00,2025-06-20 12:35:00,6,89995.093764,91625.0,None
7,2025-06-20 12:05:00,2025-06-20 12:40:00,7,90050.275277,91962.0,None
8,2025-06-20 12:05:00,2025-06-20 12:45:00,8,90098.543934,92302.0,None
9,2025-06-20 12:05:00,2025-06-20 12:50:00,9,90141.771018,92640.0,None


In [ ]:
df_new = pd.DataFrame([r.__dict__ for r in rec])
df_preds = pd.concat([df_preds, df_new], ignore_index=True)